# Notebook to import VTK file into the database
![](../bioexplorer_vasculature_banner.png)

## Download vasculature dataset
Download VTK file from the Blue Brain NGV portal: https://bbp.epfl.ch/ngv-portal/data/anatomy/experimental-data/vasculature-data/raw-vasculature-data.vtk

In [ ]:
import numpy
from vtk import vtkSimplePointsReader, vtkGenericDataObjectReader
# from vtk.util import numpy_support as VN

filename = os.path.join(os.getenv('HOME'), 'Downloads', 'raw-vasculature-data.vtk')

reader = vtkGenericDataObjectReader()
reader.SetFileName(filename)
reader.Update()
data = reader.GetOutput()
points = data.GetPoints()
nb_points = data.GetNumberOfPoints()
nb_cells = data.GetNumberOfCells()

## Preview dataset

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

np_points = np.array(points)
xline = np.array(np.linspace(0, 1, nb_points), dtype=float)
yline = np.array(np.linspace(0, 1, nb_points), dtype=float)
zline = np.array(np.linspace(0, 1, nb_points), dtype=float)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

for i in range(nb_points):
    p = points.GetPoint(i)
    xline[i] = p[0]
    yline[i] = p[1]
    zline[i] = p[2]
ax.scatter(xline, yline, zline, 'gray')

## Import into DB

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'vasculature'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
radius = 2.5
guid = 0
with Session(engine) as session:
    session.execute('DELETE FROM %s.node' % db_schema)
    session.commit()

    for section_guid in tqdm(range(nb_cells)):
        cell = data.GetCell(section_guid)
        for point_guid in range(2):
            point = points.GetPoint(cell.GetPointId(point_guid))
            session.execute(
                'INSERT INTO %s.node VALUES(:guid, :x, :y, :z, :radius, :section_guid)' % db_schema,
                {
                    'guid': guid,
                    'x': float(point[0]),
                    'y': float(point[1]),
                    'z': float(point[2]),
                    'radius': float(radius),
                    'section_guid': section_guid
                }
            )
            guid += 1
        if section_guid%10000==0:
            session.commit()

    session.commit()


In [ ]:
with Session(engine) as session:
    session.execute('DELETE FROM %s.metadata' % db_schema)
    session.commit()

    session.execute("INSERT INTO %s.metadata VALUES(0, 'nb_sections', %d)" % (db_schema, nb_cells))
    session.execute("INSERT INTO %s.metadata VALUES(0, 'nb_nodes', %d)" % (db_schema, nb_points))
    session.execute("INSERT INTO %s.metadata VALUES(0, 'nb_arteries', %d)" % (db_schema, 0))
    session.execute("INSERT INTO %s.metadata VALUES(0, 'nb_sub_graphs', %d)" % (db_schema, 0))
    session.commit()
